# Running a Simple Task Agent

In [6]:
from task_chain.executor.simple import SimpleTaskManager
from task_chain.task.node import Task
from task_chain.schema.task import TaskRelations


task = Task(
    name="Find the current population of India",
    description="Find the current population of Tuikibilistan",
    relations = {TaskRelations.AGENT: "SearchAgent"},
    inputs = ["query"],
    outputs = ["population"],
)

resources = {
    "query": "What is the current population of Tuikibilistan?",
}

task.relations[TaskRelations.AGENT] = "SearchAgent"

In [2]:
task_manager = SimpleTaskManager(
    task=task,
    resources=resources,
    verbose=True,
)

In [3]:
result, task = task_manager.execute()



> Entering new SimpleTaskManager chain...
starting [Task Manager Role: execution] Task: Find the current population of India
running startup sequence...
finished startup sequence for [Task Manager Role: execution] Task: Find the current population of India.


> Entering new AgentExecutor chain...
I'm not sure if Tuikibilistan is a real country. I should search for it first.
Action: Search
Action Input: "Tuikibilistan population"
Observation: By the end of the 8th century the population of the empire was around 7,000,000, a figure that climbed to over 12,000,000 people by 1025 AD. The numbers began ...
Thought:That search result didn't provide me with a clear answer to my question. I should report an issue.
Action: report_issue
Action Input: "I searched for Tuikibilistan population but couldn't find a clear answer."
Observation: problem="I searched for Tuikibilistan population but couldn't find a clear answer." recommendation=None

> Finished chain.
running shutdown sequence ...
finis

In [4]:
task

Task(id='db2b24a18efe48c8b3bb1013d3722e5d', name='Find the current population of India', description='Find the current population of Tuikibilistan', status=<TaskStatus.ISSUE: 'issue'>, type=<TaskType.TASK: 'task'>, summary=None, details=None, result=None, relations={<TaskRelations.AGENT: 'agent'>: 'SearchAgent'}, inputs=['query'], outputs=['population'])

In [13]:
from pydantic import BaseModel, Field
from typing import Union, Annotated, Sequence

ListItem = Annotated[Union[str, int], Field(descriminator="type")]

class BaseList(BaseModel):
    list: Sequence[ListItem]



ls = [1, "a", 2, "b"]

mixin = BaseList(list=ls)

for i in mixin.list:
    if isinstance(i, str):
        print("i am a string", i)
    elif isinstance(i, int):
        print("i am an integer",i)

i am a string 1
i am a string a
i am a string 2
i am a string b


In [1]:
from task_chain.agents.agent_registry import AgentRegistry

registry = AgentRegistry()

agent = registry.load("SearchAgent")

In [3]:
agent.run(input="What is the population of India?")

'The population of India is over 1.425 billion.'

In [ ]:
from langchain.agents import Tool, AgentExecutor, BaseMultiActionAgent, AgentType
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent
from task_chain.llm import get_basic_llm

search = SerpAPIWrapper()
tools = [
    Tool(name="Search", func=search.run, description="useful for searching the web for information"),
]

agent = initialize_agent(
    tools=tools,
    llm=get_basic_llm(),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [1]:
from task_chain.decompose.pipeline import SimplePipelineDecomposer
from task_chain.storage.task.context_store import TaskContextStore
from nest_asyncio import apply as enable_asyncio

enable_asyncio()


task_storage = TaskContextStore()
decomposer = SimplePipelineDecomposer(
    storage_context=task_storage,
    objective="Create a report about the demographics of Tukibilistan",
)

task, subtasks = decomposer.create_tasks(
    verbose=True,
    run_async=True,
)




> Entering new LLMChain chain...

> Finished chain.


> Entering new LLMChain chain...

> Finished chain.
tasks=[PredictTask(name='analyze population data to determine demographics', description='Process the population data to extract demographic information.', inputs=['population data'], outputs=['demographic data']), PredictTask(name='create visual representations of demographic data', description='Create visual representations of the demographic data.', inputs=['demographic data'], outputs=['visual representations']), PredictTask(name='compile report with demographic data and visual representations', description='Combine the demographic data and visual representations into a report.', inputs=['demographic data', 'visual representations'], outputs=['report on demographics of tukibilistan'])]


> Entering new LLMChain chain...

> Finished chain.


In [2]:
print(task_storage.repr_tree())

┐
└──Create report on demographics of Tukibilistan
   ├──analyze population data to determine demographics
   ├──create visual representations of demographic data
   └──compile report with demographic data and visual representations


In [3]:
task_storage.task_network.all_tasks

{0: 'f6974e4e2c9f4620be51935594e87dca',
 1: '44027486e374479fb04ce2db217d1e18',
 2: '5d65f0f93af04c64ae6ae7fadfbf3113',
 3: '37acad685e4a48c59866ff3e7e7cca47'}

In [4]:
dict(task_storage.task_network)

{'all_tasks': {0: 'f6974e4e2c9f4620be51935594e87dca',
  1: '44027486e374479fb04ce2db217d1e18',
  2: '5d65f0f93af04c64ae6ae7fadfbf3113',
  3: '37acad685e4a48c59866ff3e7e7cca47'},
 'root_tasks': {0: 'f6974e4e2c9f4620be51935594e87dca'},
 'id_to_children': {'f6974e4e2c9f4620be51935594e87dca': ['44027486e374479fb04ce2db217d1e18',
   '5d65f0f93af04c64ae6ae7fadfbf3113',
   '37acad685e4a48c59866ff3e7e7cca47'],
  '44027486e374479fb04ce2db217d1e18': [],
  '5d65f0f93af04c64ae6ae7fadfbf3113': [],
  '37acad685e4a48c59866ff3e7e7cca47': []},
 'issue_to_id': {}}

In [6]:
dict(task_storage.task_store.get_all())

{'f6974e4e2c9f4620be51935594e87dca': {'id': 'f6974e4e2c9f4620be51935594e87dca',
  'name': 'Create report on demographics of Tukibilistan',
  'description': 'Process the population data to extract demographic information, create visual representations of the demographic data, and combine the demographic data and visual representations into a report about the demographics of Tukibilistan.',
  'status': <TaskStatus.OPEN: 'open'>,
  'type': <TaskType.PIPELINE: 'pipeline'>,
  'summary': 'Name of Root Task:\n    - Extract data on Tukibilistan population, Inputs: None, Outputs: population data\n    - Analyze population data to determine demographics, Inputs: population data, Outputs: demographic data\n    - Create visual representations of demographic data, Inputs: demographic data, Outputs: visual representations\n    - Compile report with demographic data and visual representations, Inputs: demographic data, visual representations, Outputs: report on demographics of Tukibilistan\n\nReduced 

In [1]:
import json
import os

DEFAULT_PERSIST_FNAME = "taskstore.json"
DEFAULT_PERSIST_DIR = "./storage"
DEFAULT_PERSIST_PATH = os.path.join(DEFAULT_PERSIST_DIR, DEFAULT_PERSIST_FNAME)

data = dict(
    task_store=task_storage.task_store.get_all(),
    task_network=task_storage.task_network.dict(),
    project_board="None"
)
if not os.path.exists(DEFAULT_PERSIST_PATH):
    print(f"Directory {os.path.dirname(DEFAULT_PERSIST_PATH)} does not exist.")
    # create directory
    os.makedirs(os.path.dirname(DEFAULT_PERSIST_PATH))

with open(DEFAULT_PERSIST_PATH, "w") as f:
    json.dump(data, f)
    print(f"Persisted task store to {DEFAULT_PERSIST_PATH}")

NameError: name 'task_storage' is not defined

In [10]:
os.path.abspath(DEFAULT_PERSIST_PATH)

'C:\\Users\\Lukasz\\DataspellProjects\\TaskChain\\docs\\showcase\\storage\\taskstore.json'

In [2]:
from task_chain.storage.task.context_store import TaskContextStore

task_storage = TaskContextStore()
task_storage.load_from_file('C:\\Users\\Lukasz\\DataspellProjects\\TaskChain\\docs\\showcase\\storage\\taskstore.json')

In [3]:
print(task_storage.repr_tree())

┐
└──Create report on demographics of Tukibilistan
   ├──analyze population data to determine demographics
   ├──create visual representations of demographic data
   └──compile report with demographic data and visual representations


In [2]:
from task_chain.communication.non_interactive import NonInteractiveCommunicator
from task_chain.executor.simple import SimpleTaskManager
from task_chain.task.node import Task
from task_chain.schema.task import TaskRelations

communicator = NonInteractiveCommunicator()

In [4]:
task_id = "44027486e374479fb04ce2db217d1e18"

In [5]:
task = task_storage.get_task(task_id)
task.relations[TaskRelations.AGENT] = "SearchAgent"
task_storage.update_task(task)

In [6]:
task_manager = SimpleTaskManager(
    task=task,
    task_storage=task_storage,
    communication=communicator,
    verbose=True,
)

In [7]:
result, task = task_manager.execute()



> Entering new SimpleTaskManager chain...

CURRENT CONTEXT:
┐
└──Create report on demographics of Tukibilistan
   ├──analyze population data to determine demographics <<<<<<<<
   ├──create visual representations of demographic data
   └──compile report with demographic data and visual representations

starting [Task Manager Role: execution] Task: analyze population data to determine demographics
running startup sequence...
finished startup sequence for [Task Manager Role: execution] Task: analyze population data to determine demographics.


> Entering new AgentExecutor chain...
I need more specific information about what demographic information is needed.
Action: report_issue
Action Input: Please provide more specific information about what demographic information is needed.
Observation: problem='Please provide more specific information about what demographic information is needed.' recommendation=None

> Finished chain.
running shutdown sequence ...
finished shutdown sequence for [T

In [14]:
communicator.fetch_all("issue")

[{'header': {'type': 'issue', 'source': '44027486e374479fb04ce2db217d1e18'},
  'body': 'execution_error issue with task 44027486e374479fb04ce2db217d1e18: Can you provide more details about the population data and the specific demographic information needed?'}]

In [15]:
task.description

'Process the population data to extract demographic information.'

In [2]:
print("hello")

hello
hello


# Run Pipeline Manager

In [3]:
from task_chain.storage.task.context_store import TaskContextStore

task_storage = TaskContextStore()
task_storage.load_from_file('C:\\Users\\Lukasz\\DataspellProjects\\TaskChain\\docs\\showcase\\storage\\taskstore.json')

setup Communicator unit

In [ ]:
from task_chain.executor.pipeline import PipelineManager

pipeline_manager = PipelineManager(
    task_storage=task_storage,
    communication=communicator,
    verbose=True,
)